In [74]:
import math
import numpy as np
import QuantLib as ql
import scipy as sp

In [75]:
mean = 9999.900983333333
std = 88.75266514702373
t = 250
S = 10000
K = 10000
r = 0.01

d1 = (math.log(S/K) + (r + (std**2)/2)*t) / (std * math.sqrt(t))
d2 = d1 - std * math.sqrt(t)

def N(x):
    return sp.stats.norm.cdf(x, mean, std)
    return 0.5 * (1 + math.erf((x-mean) / (std*math.sqrt(2))))

print(d1)
print(d2)
# print(N(d1) * S)
# print(N(d2) * K * math.e**(-r*t))

# N = norm.cdf
def BS_CALL(S, K, T, r, sigma):
    d1 = (np.log(S/K) + (r + sigma**2/2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return S * N(d1) - K * np.exp(-r*T)* N(d2)

# C = N(d1) * S - N(d2) * K * math.e**(-r*t)
# C

print(BS_CALL(S, K, t, r, std))

701.6532071984752
-701.6496441757142
0.0


In [76]:
def option_price_european(
S0, K, days_to_maturity, r, sigma, call=True, engine="BS", steps=200, paths=10000):
    """
    This function returns the price of a European Call with 3 different price␣
    ↪engines: BS, BT and MC.
    """
    # Setting up the dates
    today = ql.Date().todaysDate()
    maturity = today + days_to_maturity
    # Choosing the type of the option
    if call:
        option_type = ql.Option.Call
    else:
        option_type = ql.Option.Put
    # Creating the payoff
    payoff = ql.PlainVanillaPayoff(option_type, K)
    europeanExercise = ql.EuropeanExercise(maturity)
    europeanOption = ql.VanillaOption(payoff, europeanExercise)
    # Setting up the option's characteristics
    spot = ql.SimpleQuote(S0)
    riskFreeTS = ql.YieldTermStructureHandle(
    ql.FlatForward(today, r, ql.Actual365Fixed())
    )
    volTS = ql.BlackVolTermStructureHandle(
    ql.BlackConstantVol(today, ql.NullCalendar(), sigma, ql.Actual365Fixed())
    )
    # We will use the analytical way of pricing the options
    process = ql.BlackScholesProcess(ql.QuoteHandle(spot), riskFreeTS, volTS)
    if engine == "BS":
        engine = ql.AnalyticEuropeanEngine(process)
    elif engine == "BT":
        engine = ql.BinomialVanillaEngine(process, "crr", steps)
    elif engine == "MC":
        engine = ql.MCEuropeanEngine(
        process, "PseudoRandom", timeSteps=steps, requiredSamples=paths
    )
    europeanOption.setPricingEngine(engine)
    return europeanOption.NPV()

In [80]:
option_price_european(S, K, t, 0.05, 0.2, call=True, engine="BS", steps=200, paths=10000)

830.6094887461612